In [1]:
from jinja2 import Environment, FileSystemLoader
import pandas as pd
import geopandas as gpd
pd.options.display.float_format = '{:,.2f}'.format
import plotly.express as px

In [2]:
pc = "Corsham"

In [3]:
gdf = gpd.read_file("G:/OS_OpenData/OS_BoundaryLine/OS_BL_Parish_Wilts.shp")
# Create rank for PC area
gdf['AreaPC'] = gdf['HECTARES']
df_pc_sorted = gdf.sort_values(by = "AreaPC", ascending=False)
df_pc_sorted['rankArea'] = list(range(1, df_pc_sorted.shape[0]+1))

In [4]:
dfpc = df_pc_sorted[df_pc_sorted.NAME == pc]
pcAreaRank =dfpc['rankArea'].item()

In [5]:
df = pd.read_csv("data/outputData.csv")


df_pc = df[df.NAME == pc]

In [6]:
df_pc_sorted = df_pc.sort_values(by = "rank", ascending=True)
df_pc_filter = df_pc_sorted[df_pc_sorted['rank']<=10]

In [7]:
df_pc_filter = df_pc_filter[df_pc_filter.datasetName.isin(['Long-Established Woodland and Wood Pasture','National Forest Inventory','Greenspace','PHI','WSBRC Priority Habitats'])]


In [8]:
pcAreaHa = df_pc['AreaPC'].iloc[0].astype(int)


In [16]:
df_pc['unit'].unique()

array(['Ha', 'Metres'], dtype=object)

In [45]:
df_pc_filter_dataset

,Unnamed: 0.1,NAME,value,valuePercent,groupColumnValue,groupColumnName,mapGroup,mapName,rank,rankPercent,unitName,unit,datasetName,source,Unnamed: 0,AreaPC
744,798,Corsham,17,0.66,Total,Total,Woodland,NaN,24,45,COUNT,COUNT,Ancient Tree Inventory,WT,0,"2,575.21"


In [52]:
df_small

,Total,COUNT
11806,Total,2


In [14]:
table_dict = {}
for category in df_pc.mapGroup:
    dataset_dict = {}
    df_pc_category =  df_pc[df_pc.mapGroup == category]
    
    for dataset in df_pc_category.datasetName:
        if df_pc_category.shape[0] >0:
            print(dataset)
            df_pc_filter_dataset = df_pc_category[df_pc_category.datasetName == dataset]
            # Check if dataset is an area, and if it is, include the value percent
            if df_pc_filter_dataset['unit'].iloc[0] == "Ha":
                df_small = df_pc_filter_dataset[['groupColumnValue','value', 'valuePercent']]
                
                df_small = df_small.rename(columns = {'groupColumnValue':df_pc_filter_dataset['groupColumnName'].iloc[0],
                                                    "value":df_pc_filter_dataset['unit'].iloc[0],
                                                    "valuePercent":"Percentage of PC"})
            # If dataset in metres or count, do not include valuePercent as is not relevant
            elif df_pc_filter_dataset['unit'].iloc[0].lower() == "metres" or df_pc_filter_dataset['unit'].iloc[0].lower() == "count" :
                df_small = df_pc_filter_dataset[['groupColumnValue','value']]
                
                df_small = df_small.rename(columns = {'groupColumnValue':df_pc_filter_dataset['groupColumnName'].iloc[0],
                                                    "value":df_pc_filter_dataset['unit'].iloc[0]})
                
            # Check if column value has more than one item, if not, do not include figure as no point showing on graph
            if df_small[df_pc_filter_dataset['groupColumnName'].iloc[0]].unique().shape[0]==1:
                dataset_dict[dataset] = {"table":df_small.to_html(index = False),
                                     "fig":""
                                     }
            # If column value as more than one item, add in figure. 
            else: 
                fig = px.bar(df_small, x = df_pc_filter_dataset['groupColumnName'].iloc[0],y =df_pc_filter_dataset['value'], template="plotly_white", height = 350 )
                fig.update_layout(
                    margin={'t':0,'l':0,'b':0,'r':0}
                )
                dataset_dict[dataset] = {"table":df_small.to_html(index = False),
                                     "fig":""#fig.to_html(full_html=False)
                                     }
    table_dict[category] = dataset_dict

Landscape Character Areas
Landscape Character Areas
Landscape Character Areas
Landscape Character Areas
Landscape Character Areas
Landscape Character Areas
Landscape Character Areas
Landscape Character Areas
Landscape Character Areas
Ancient Tree Inventory
National Forest Inventory
National Forest Inventory
National Forest Inventory
National Forest Inventory
National Forest Inventory
National Forest Inventory
National Forest Inventory
National Forest Inventory
National Forest Inventory
Long-Established Woodland and Wood Pasture
Long-Established Woodland and Wood Pasture
Long-Established Woodland and Wood Pasture
Long-Established Woodland and Wood Pasture
Long-Established Woodland and Wood Pasture
Bedrock
Bedrock
Faultlines
Bedrock
Bedrock
Faultlines
Bedrock
Bedrock
Faultlines
Lakes Inventory Councils
Flood Alert Areas
Main Rivers
Risk of Flooding
Lakes Inventory Councils
Flood Alert Areas
Main Rivers
Risk of Flooding
Lakes Inventory Councils
Flood Alert Areas
Main Rivers
Risk of Floodi

In [71]:
table_dict.keys()

dict_keys(['LCA', 'Woodland', 'Geology', 'Water Features', 'Greenspace', 'PHI', 'WSBRC Priority Habitats', 'LBAs', 'Access & Routeways', 'Heritage', 'Protected Areas', 'Priority Habitat'])

In [67]:
table_dict.keys()

dict_keys(['LCA', 'Woodland', 'Geology', 'Water Features', 'Greenspace', 'PHI', 'WSBRC Priority Habitats', 'LBAs', 'Access & Routeways', 'Heritage', 'Protected Areas', 'Priority Habitat'])

In [18]:
environment = Environment(loader=FileSystemLoader("templates/"))
template = environment.get_template("Stats template.html")


filename = f"outputReports/{pc}_stats.html"
# filename = f"outputReports/test_stats.html"

content = template.render(
    pc = pc,
    pcAreaHa = format(pcAreaHa.round(-1),","),
    pcAreaRank = pcAreaRank,
    scores = df_pc_filter,
    data = table_dict

)
with open(filename, mode="w", encoding="utf-8") as message:
    message.write(content)
    print(f"... wrote {filename}")

... wrote outputReports/Corsham_stats.html


In [79]:
template

<Template 'Stats template.html'>

In [52]:
pcAreaRank = 2
test_name = "Python Challenge"
students = [
    {"name": "Sandrine",  "score": 100},
    {"name": "Gergeley", "score": 87},
    {"name": "Frieda", "score": 92},
]

environment = Environment(loader=FileSystemLoader("templates/"))
template = environment.get_template("test1.txt")

for student in students:
    filename = f"message_{student['name'].lower()}.txt"
    content = template.render(
        student,
        pcAreaRank=pcAreaRank,
        test_name=test_name
    )
    with open(filename, mode="w", encoding="utf-8") as message:
        message.write(content)
        print(f"... wrote {filename}")

... wrote message_sandrine.txt
... wrote message_gergeley.txt
... wrote message_frieda.txt
